# 🚀 OpenClaw + Dolphin 3 - Google Colab

Sansürsüz AI Sunucusu - Colab'da Çalıştır

## 📋 Adımlar

1. **Runtime → Change runtime type → T4 GPU** seç
2. Tüm hücreleri sırayla çalıştır
3. Ngrok token'ını gir
4. Public URL'yi kopyala
5. Tarayıcıda aç ve kullan!

## ⏱️ Kurulum Süresi: ~15-20 dakika

In [ ]:
# 1. Gerekli paketleri kur
print("📦 Paketler kuruluyor...")
!pip install -q vllm==0.2.7 flask pyngrok requests

In [ ]:
# 2. app.py dosyasını GitHub'dan indir
print("📥 app.py indiriliyor...")
!wget -q https://raw.githubusercontent.com/SuperHelix77/openclaw-dolphin3/main/app.py -O app.py
print("✅ app.py indirildi")

In [ ]:
# 3. Flask sunucusunu arka planda başlat
import subprocess
import time
import threading

print("🚀 Flask sunucusu başlatılıyor...")
print("⏳ Model yükleniyor (15-20 dakika sürebilir)...")

def run_flask():
    subprocess.run(["python", "app.py"])

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Sunucunun başlaması için bekle
time.sleep(5)
print("✅ Flask sunucusu başlatıldı (arka planda çalışıyor)")

In [ ]:
# 4. Ngrok ile public URL oluştur
from pyngrok import ngrok
import getpass

print("🔑 Ngrok token gerekli")
print("👉 https://dashboard.ngrok.com/get-started/your-authtoken")
print("")

ngrok_token = getpass.getpass("Ngrok token'ını yapıştır: ")
ngrok.set_auth_token(ngrok_token)

print("\n🌐 Public URL oluşturuluyor...")
public_url = ngrok.connect(8000)

print("\n" + "="*70)
print("✅ BAŞARILI! Sunucu hazır.")
print("="*70)
print(f"\n🌍 Public URL: {public_url}")
print("\n📝 Bu URL'yi tarayıcıda aç ve kullan!")
print("\n⚠️  Not: Colab sekmesini AÇIK TUT, yoksa sunucu kapanır.")
print("="*70)

In [ ]:
# 5. Sunucu loglarını izle (isteğe bağlı)
print("📊 Sunucu logları (Ctrl+C ile durdur)")
print("="*70)
!tail -f /dev/null  # Sunucu çalışmaya devam etsin

## 🎉 Tamamlandı!

Artık web arayüzünü kullanabilirsin:

1. Yukarıdaki **Public URL**'yi kopyala
2. Yeni bir tarayıcı sekmesinde aç
3. Chat arayüzünü kullan!

## 📱 OpenClaw Config

Yerel makinende `~/.openclaw/config.yaml`:

```yaml
models:
  default:
    provider: openai
    model: dolphin-2.9-llama3.1-8b
    credentials:
      apiKey: 'ollama'
      apiBase: 'YOUR_PUBLIC_URL_HERE'
```

## ⚠️ Önemli Notlar

- Colab sekmesini **AÇIK TUT**
- 12 saat sonra otomatik kapanır
- GPU kullanımı sınırlıdır
- Ücretsiz ve sansürsüz! 🎉